In [1]:
import sqlite3
import pandas as pd
import time
import requests
from nba_api.stats.static import teams
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam
import numpy as np
import json
import difflib
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import timedelta, datetime
from src.Data.maper import EQUIPOS_ODDS

In [2]:
def scroll_to_bottom(driver, pause_time=2):
    """Desplazarse poco a poco hasta el final de la página."""
    last_height = driver.execute_script("return document.body.scrollHeight*0.9")
    
    while True:
        # Desplazarse hacia abajo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.9);")
        
        # Esperar a que se cargue el contenido
        time.sleep(pause_time)
        
        # Calcular la nueva altura de la página y comparar con la última altura
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_urls(url, pages):
    partidos = []
    home_team = []
    away_team = []
    fechas = []
    i = 0

    
    print(url)
    temp_urls = [url+f'/#/page/{j}/' for j in range(1,pages+1)]
    temp_urls = ['https://www.oddsportal.com/basketball/usa//nba-2017-2018/results/#/page/27/']
    options = Options()
    #options.add_experimental_option("detach", True)
    options.add_argument("--disable-search-engine-choice-screen")

    j = 0
    while j <= 0:
        print(temp_urls[j])
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver.get(temp_urls[j])

        driver.maximize_window()
        time.sleep(5)
        scroll_to_bottom(driver, pause_time=3)
        time.sleep(5)
        html = driver.page_source
        
        soup = BeautifulSoup(html, 'html.parser')
        
        divs = soup.find_all('div', class_='eventRow flex w-full flex-col text-xs')

        for div in divs:
            
            fecha_div = div.find('div', class_='text-black-main font-main w-full truncate text-xs font-normal leading-5')
            
            if fecha_div is not None:
                fecha_div = fecha_div.text
                fecha = fecha_div.split('-')[0].rstrip()
                if 'Today' in fecha:
                    fecha = datetime.now().strftime("%d %b %Y")
                elif 'Yesterday' in fecha:
                    fecha = (datetime.now() - timedelta(days=1)).strftime("%d %b %Y")
            divs2 = div.find_all('a', class_='next-m:flex next-m:!mt-0 ml-2 mt-2 min-h-[32px] w-full hover:cursor-pointer')

            for div2 in divs2:
                
                partidos.append('https://www.oddsportal.com' + div2['href'])
                equipos = div2.find_all('p', class_='participant-name truncate')
                home_team.append(equipos[0].text)
                away_team.append(equipos[1].text)
                fechas.append(fecha)
        j += 1        
        driver.quit()    

    temp_data_df = pd.DataFrame()       
    temp_data_df['URL'] = partidos
    temp_data_df['H_TEAM_NICKNAME'] = home_team
    temp_data_df['A_TEAM_NICKNAME'] = away_team
    temp_data_df['GAME_DATE'] = fechas

    return temp_data_df

def get_game_id(dataset_urls: pd.DataFrame):
    dataset_urls['H_TEAM_ID'] = dataset_urls['H_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
    dataset_urls['A_TEAM_ID'] = dataset_urls['A_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
    dataset_urls['GAME_DATE'] = pd.to_datetime(dataset_urls['GAME_DATE'])

    conexion = sqlite3.connect("NBA_DATA.db")

    # Crear una lista para almacenar los GAME_IDs
    game_ids = []
    # Realizar una consulta SQL para cada fila en dataset_urls
    for index, row in dataset_urls.iterrows():
        if row['H_TEAM_ID'] is None or row['A_TEAM_ID'] is None:
            game_ids.append(None)
            continue
        home_team_id = row['H_TEAM_ID']
        away_team_id = row['A_TEAM_ID']
        game_date = row['GAME_DATE']
        start_date = (game_date - pd.Timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')
        game_date = game_date.strftime('%Y-%m-%d %H:%M:%S')


        query = """
        SELECT GAME_ID, GAME_DATE
        FROM GAMES 
        WHERE H_TEAM_ID = ? 
        AND A_TEAM_ID = ? 
        AND GAME_DATE BETWEEN ? AND ?
        """
        
        result = pd.read_sql_query(query, conexion, params=[home_team_id, away_team_id, start_date, game_date])
        
        if not result.empty:
            result.sort_values(by='GAME_DATE', ascending=False, inplace=True)
            if len(result) > 1:
                game_ids.append(result.loc[result["GAME_DATE"] == game_date]['GAME_ID'])
            else:
                game_ids.append(result.iloc[0]['GAME_ID'])
        else:
            game_ids.append(None)
    dataset_urls['GAME_ID'] = game_ids

    return dataset_urls[['URL', 'H_TEAM_NICKNAME', 'A_TEAM_NICKNAME', 'GAME_DATE', 'GAME_ID']]

def get_season(url: str):
    conexion = sqlite3.connect("NBA_DATA.db")
    season = url.split('/')[5].split('-')[1:]
    season = season[0] + '-'+season[1][2:]

    query = """
        SELECT GAME_ID
        FROM GAMES
        WHERE SEASON = ?
        """
    
    return pd.read_sql_query(query, conexion, params=[season])

def get_odds(dataset_urls: pd.DataFrame):

    keys = ['URL', 'GAME_ID', 'H_TEAM_NICKNAME', 'A_TEAM_NICKNAME', 'GAME_DATE', 'Average_1','Average_X','Average_2','Highest_1','Highest_X','Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X','Average_12','Average_X2','Highest_1X','Highest_12','Highest_X2']
    extensiones = ['#1X2;3', '#home-away;1', '#double;3']
    columnas = [['Average_1','Average_X','Average_2','Highest_1','Highest_X','Highest_2'],['Average_H', 'Average_A', 'Highest_H', 'Highest_A'],
            ['Average_1X','Average_12','Average_X2','Highest_1X','Highest_12','Highest_X2']]

    dataset_urls = get_game_id(dataset_urls)
    ids = list(get_season(dataset_urls.iloc[-1]['URL'])['GAME_ID'])
    filtered_dataset_urls = dataset_urls[~dataset_urls['GAME_ID'].isin(ids)]
    filtered_dataset_urls = dataset_urls[(~dataset_urls['GAME_ID'].isin(ids)) & (dataset_urls['GAME_ID'].notna())]
    
    options = Options()
    #options.add_experimental_option("detach", True)
    options.add_argument("--disable-search-engine-choice-screen")
    options.add_argument("--headless")
    
    dataset_ods = pd.DataFrame(columns=keys)

    for index, row in filtered_dataset_urls.iterrows():

        print(index)
        diccionario_estadisticas = {key: None for key in keys}
        diccionario_estadisticas['URL'] = row['URL']
        diccionario_estadisticas['H_TEAM_NICKNAME'] = row['H_TEAM_NICKNAME']
        diccionario_estadisticas['A_TEAM_NICKNAME'] = row['A_TEAM_NICKNAME']
        diccionario_estadisticas['GAME_DATE'] = row['GAME_DATE']
        diccionario_estadisticas['GAME_ID'] = row['GAME_ID']
        i = -1
        for extension in extensiones:
            
            try:
                i += 1
                url = row['URL'] + extension

                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

                driver.get(url)
                driver.maximize_window()
                time.sleep(3)
                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(3)  # Espera un poco para que se cargue el contenido dinámico
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                    
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')

                divs = soup.find_all('div', class_='border-black-borders flex h-9 border-b border-l border-r text-xs')

                for div in divs:
                    ps = div.find_all('p', class_='height-content')
                    calculator_list = [[] for _ in range(len(ps)-1)]
                    for i in range(len(ps)-1):
                        calculator_list[i].append(float(ps[i+1].text))
                from statistics import mean
                medias = [mean(lista) for lista in calculator_list]

                # Aplicar la transformación según la condición
                for j in range(len(medias)):
                    diccionario_estadisticas[columnas[i][j]] = medias[j]
            except Exception as e:
                # Si ocurre un error, se captura la excepción y se imprime la variable
                print(f"Ocurrió un error: {e} en el partido {index}")

        data_df = pd.DataFrame([diccionario_estadisticas])
        #data_df[['GAME_ID', 'Average_1', 'Average_X', 'Average_2', 'Highest_1', 'Highest_X', 'Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X', 'Average_12', 'Average_X2', 'Highest_1X', 'Highest_12', 'Highest_X2']].to_sql('GAME_ODS', conexion, if_exists='append', index=False)
        # Aqui debo incluir el formateo para que todo funcione correctamente
        dataset_ods = pd.concat([dataset_ods, data_df], ignore_index=True)
    return dataset_ods

def process_odds(dataset_ods: pd.DataFrame):
    conexion = sqlite3.connect("NBA_DATA.db")

    query = """
    SELECT GAME_ID
    FROM GAME_ODS
    """
    result = pd.read_sql_query(query, conexion)
    dataset_ods = dataset_ods[~dataset_ods['GAME_ID'].isin(list(result['GAME_ID']))]

    # Cerrar la conexión
    conexion.close()
    return dataset_ods[['GAME_ID', 'Average_1', 'Average_X', 'Average_2', 'Highest_1', 'Highest_X', 'Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X', 'Average_12', 'Average_X2', 'Highest_1X', 'Highest_12', 'Highest_X2']]


In [3]:
x = get_urls('a', 1)

a
https://www.oddsportal.com/basketball/usa//nba-2017-2018/results/#/page/27/


In [4]:
dataset_urls = get_game_id(x)
ids = list(get_season(dataset_urls.iloc[-1]['URL'])['GAME_ID'])
filtered_dataset_urls = dataset_urls[~dataset_urls['GAME_ID'].isin(ids)]
filtered_dataset_urls = dataset_urls[(~dataset_urls['GAME_ID'].isin(ids)) & (dataset_urls['GAME_ID'].notna())]
dataset_urls[(~dataset_urls['GAME_ID'].isin(ids))]

,URL,H_TEAM_NICKNAME,A_TEAM_NICKNAME,GAME_DATE,GAME_ID
14,https://www.oddsportal.com/basketball/usa/nba-...,Golden State Warriors,Sacramento Kings,2017-10-14,None
15,https://www.oddsportal.com/basketball/usa/nba-...,Los Angeles Clippers,Los Angeles Lakers,2017-10-14,None
16,https://www.oddsportal.com/basketball/usa/nba-...,Milwaukee Bucks,Detroit Pistons,2017-10-14,None
17,https://www.oddsportal.com/basketball/usa/nba-...,Philadelphia 76ers,Miami Heat,2017-10-14,None
18,https://www.oddsportal.com/basketball/usa/nba-...,Chicago Bulls,Toronto Raptors,2017-10-14,None
19,https://www.oddsportal.com/basketball/usa/nba-...,Houston Rockets,San Antonio Spurs,2017-10-14,None
20,https://www.oddsportal.com/basketball/usa/nba-...,Memphis Grizzlies,New Orleans Pelicans,2017-10-14,None
21,https://www.oddsportal.com/basketball/usa/nba-...,New York Knicks,Washington Wizards,2017-10-14,None
22,https://www.oddsportal.com/basketball/usa/nba-...,Orlando Magic,Cleveland Cavaliers,2017-10-14,None
23,https://www.oddsportal.com/basketball/usa/nba-...,Charlotte Hornets,Dallas Mavericks,2017-10-14,None


In [5]:
x_ods = get_odds(x)
x_ods

,URL,GAME_ID,H_TEAM_NICKNAME,A_TEAM_NICKNAME,GAME_DATE,Average_1,Average_X,Average_2,Highest_1,Highest_X,...,Average_H,Average_A,Highest_H,Highest_A,Average_1X,Average_12,Average_X2,Highest_1X,Highest_12,Highest_X2


In [6]:
dataset_urls = get_game_id(x)
filas_con_nan = dataset_urls.isna().any(axis=1)

# Mostrar solo las filas que tienen al menos un NaN
df_con_nan = dataset_urls[filas_con_nan]
df_con_nan

,URL,H_TEAM_NICKNAME,A_TEAM_NICKNAME,GAME_DATE,GAME_ID
14,https://www.oddsportal.com/basketball/usa/nba-...,Golden State Warriors,Sacramento Kings,2017-10-14,None
15,https://www.oddsportal.com/basketball/usa/nba-...,Los Angeles Clippers,Los Angeles Lakers,2017-10-14,None
16,https://www.oddsportal.com/basketball/usa/nba-...,Milwaukee Bucks,Detroit Pistons,2017-10-14,None
17,https://www.oddsportal.com/basketball/usa/nba-...,Philadelphia 76ers,Miami Heat,2017-10-14,None
18,https://www.oddsportal.com/basketball/usa/nba-...,Chicago Bulls,Toronto Raptors,2017-10-14,None
19,https://www.oddsportal.com/basketball/usa/nba-...,Houston Rockets,San Antonio Spurs,2017-10-14,None
20,https://www.oddsportal.com/basketball/usa/nba-...,Memphis Grizzlies,New Orleans Pelicans,2017-10-14,None
21,https://www.oddsportal.com/basketball/usa/nba-...,New York Knicks,Washington Wizards,2017-10-14,None
22,https://www.oddsportal.com/basketball/usa/nba-...,Orlando Magic,Cleveland Cavaliers,2017-10-14,None
23,https://www.oddsportal.com/basketball/usa/nba-...,Charlotte Hornets,Dallas Mavericks,2017-10-14,None
